In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.6MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [10]:
cd "/content/drive/My Drive/Colab Notebooks/transformacja2020"

/content/drive/My Drive/Colab Notebooks/transformacja2020


In [0]:
df= pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [22]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0] #mapowanie - zmienia wszystkie litery na małe
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [23]:
model =   RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0) #n_est ilosc drzew
run_model(['brand_cat'], model)


(-57.31783843165656, 4.181246596160967)

In [24]:
df.features.head().values #sa w strngu chcemy je przemapowac do slownika  można użyć: literal_eval(str_dict) gdzie str_dict to string 

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x)  == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for iteam in features:
    key = iteam['key'].lower().strip() #strip usuwa spacje
    value = iteam['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [36]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [38]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )

In [39]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_item style', 'feat_age segment', 'feat_shape',
       'feat_style number', 'feat_water-resistant', 'feat_fishing',
       'feat_wheel type', 'feat_construction', 'feat_product #',
       'feat_issued/ not-issued'],
      dtype='object', length=526)

In [40]:
df[ False == df['feat_athlete'].isnull() ].shape[0] #zlicza ile razy wystapila cecha feat_athlete jesli jest mało danej cechy nie warto jej brać pod uwagę

8

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df [False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0]*100

In [52]:
{k:v for k,v in keys_stat.items() if v > 30} # wyrzuca tylko te atrybuty które występują x% w całości

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

In [56]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape #sprawedza ile jest takich samych nazw  w kolumanch przed i po normalizacji

(8846, 531)

In [59]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
feats =['brand_cat','feat_color_cat', 'feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat']
run_model(feats,model)

(-57.39075514016982, 4.306045631189478)

In [91]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
print (results)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)  #pokazuje ktore cechy sa najmocniesze 


(-57.32590245751736, 4.183082952181771)


Weight,Feature
0.2587 ± 0.0127,brand_cat
0.1040 ± 0.0132,feat_material_cat
0.0259 ± 0.0018,feat_gender_cat
0.0125 ± 0.0009,feat_shape_cat
0.0058 ± 0.0013,feat_metal type_cat
0.0034 ± 0.0006,feat_ring size_cat
0.0026 ± 0.0005,feat_style_cat
0.0023 ± 0.0004,feat_occasion_cat
0.0002 ± 0.0000,feat_sport_cat


In [0]:
feats = ['brand_cat', 'feat_gender_cat','feat_material_cat', 'feat_sport_cat','feat_style_cat', 'feat_metal type_cat','feat_ring size_cat','feat_shape_cat','feat_occasion_cat']

feats = list(set(feats)) #zostawia tyle niepowtarzajace sie kolumny
model = RandomForestRegressor(max_depth=5, n_estimators=100)

results = run_model(feats,model)

In [72]:
df['brand'].value_counts(normalize=True) #najczestrze cechy danej zmiennej 

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
cabela's          0.000055
armitron          0.000055
dragonforce       0.000055
1901              0.000055
the felt store    0.000055
Name: brand, Length: 1732, dtype: float64

In [82]:
df[ df['brand'] ==  'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
!echo 'data' > .gitignore #nie wrzuca na gita katalogu data
!git add .gitignore

In [97]:
!git add day5.ipynb
!git commit -m "Read Men's Shoe Prices dataset Random Forest model"
!git config --global user.email "jedrek89@gmail.com"
!git config --global user.name "jedrzej00"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@560fa3059ca7.(none)')


In [95]:
ls


ColabDriveGithub.ipynb  day4.ipynb       hellogithub.ipynb
data/                   day4_meta.ipynb  LICENSE
day3.ipynb              day5.ipynb       README.md
